# Robust Neaural Network Training

## Imports and setups

In [1]:
from __future__ import print_function
from __future__ import division
from builtins import range
from builtins import int
from builtins import dict


import argparse

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.backends.cudnn as cudnn

import torch.nn.functional as F

import torchvision.datasets as dset
import torchvision.transforms as T

import itertools
import os

sns.set_style("darkgrid")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Loader class

In [2]:
class ChunkSampler(sampler.Sampler):
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

def loadData():

    transform = T.Compose([T.ToTensor()])

    MNIST_train = dset.MNIST('./Third Party/Robust-NN-Training/dataset', train=True, transform=T.ToTensor(), download=True)

    MNIST_test = dset.MNIST('./Third Party/Robust-NN-Training/dataset', train=False, transform=T.ToTensor(), download=True)


    loader_train = DataLoader(MNIST_train, batch_size=64)

    loader_test = DataLoader(MNIST_test, batch_size=64)

    return loader_train, loader_test

## Model

In [3]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.fc1 = nn.Linear(800, 500)
        self.fc2 = nn.Linear(500, 10)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(2)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv2d)):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 1 / m.bias.numel())
            if isinstance(m, (nn.Linear)):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 1 / m.bias.numel())

    def forward(self, x):
        x = self.maxpool(self.relu(self.conv1(x)))
        x = self.maxpool(self.relu(self.conv2(x)))
        x = x.view(-1, 800)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x

## Loss Function

In [4]:
def loss_function(model, X, y, dtype, eps, MaxIter_max, step_size_max):

    N = X.shape[0]
    X = X.repeat(1, 10, 1, 1).reshape(N * 10, 1, 28, 28)
    X_copy = X.clone()
    X.requires_grad = True

    y = y.view(-1, 1).repeat(1, 10).view(-1, 1).long().cuda()

    index = torch.tensor([jj for jj in range(10)] * N).view(-1, 1).cuda().long()

    for i in range(MaxIter_max):
        output = model(X)

        maxLoss = (output.gather(1, index) - output.gather(1, y)).mean()
        X_grad = torch.autograd.grad(maxLoss, X, retain_graph=True)[0]
        X = X + X_grad.sign() * step_size_max

        X.data = X_copy.data + (X.data - X_copy.data).clamp(-eps, eps)
        X.data = X.data.clamp(0, 1)

    preds = model(X)

    loss = (-F.log_softmax(preds)).gather(1, y.view(-1, 1)).view(-1, 10).max(dim=1)[0].mean()

    return loss

## Training Process

In [5]:
def train(loader_train, loader_test, dtype, settings):

    model = ConvNet()
    model = model.type(dtype)
    model.train()

    loss_list = []

    learning_rate = 5e-4

    num_epochs, eps, iter, stepsize = settings
	
    print('\nTraining %d epochs with learning rate %.7f' % (num_epochs, learning_rate))

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):

        print('\nTraining epoch %d / %d ...\n' % (epoch + 1, num_epochs))

        for i, (X_, y_) in enumerate(loader_train):

            X = Variable(X_.type(dtype), requires_grad=False)
            y = Variable(y_.type(dtype), requires_grad=False)

            loss = loss_function(model, X, y, dtype, eps, iter, stepsize)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % 100 == 0:
                print('Batch %d done, loss = %.7f' % (i + 1, loss.item()))

                loss_list.append(loss.item())
                test(model, loader_test, dtype)

        print('Batch %d done, loss = %.7f' % (i + 1, loss.item()))

        loss_list.append(loss.item())


    return model, loss_list

## Training quality test

In [6]:
def test(model, loader_test, dtype):
    num_correct = 0
    num_samples = 0
    model.eval()
    for X_, y_ in loader_test:

        X = Variable(X_.type(dtype), requires_grad=False)
        y = Variable(y_.type(dtype), requires_grad=False).long()

        logits = model(X)
        _, preds = logits.max(1)

        num_correct += (preds == y).sum()
        num_samples += preds.size(0)

    accuracy = float(num_correct) / num_samples * 100
    print('\nAccuracy = %.2f%%' % accuracy)
    model.train()

## Adversarial Attacks

### Projected Gradient Descent Attack

In [7]:
def pgdAttackTest(model, loader_test, dtype):

    model.eval()
    epss = [0.0, 0.1, 0.2, 0.3, 0.4]
    MaxIter = 40
    step_size = 1e-2

    for eps in epss:

        num_correct = 0
        num_samples = 0


        for X_, y_ in loader_test:

            X = Variable(X_.type(dtype), requires_grad=True)
            X_original = Variable(X_.type(dtype), requires_grad=False)
            y = Variable(y_.type(dtype), requires_grad=False).long()

            for i in range(MaxIter):
                logits = model(X)
                loss = F.cross_entropy(logits, y)
                loss.backward()

                with torch.no_grad():
                    X.data = X.data + step_size * X.grad.sign()
                    X.data = X_original + (X.data - X_original).clamp(min=-eps, max=eps)
                    X.data = X.data.clamp(min=0, max=1)
                    X.grad.zero_()

            X.requires_grad = False
            X = (X * 255).long().float() / 255

            logits = model(X)
            _, preds = logits.max(1)

            num_correct += (preds == y).sum()
            num_samples += preds.size(0)

        accuracy = float(num_correct) / num_samples * 100
        print('\nAttack using PGD with eps = %.3f, accuracy = %.2f%%' % (eps, accuracy))

###  Fast Gradient Sign Method Attack

In [8]:
def fgsmAttackTest(model, loader_test, dtype):

    model.eval()
    epss = [0.0, 0.1, 0.2, 0.3, 0.4]

    for eps in epss:

        num_correct = 0
        num_samples = 0


        for X_, y_ in loader_test:

            X = Variable(X_.type(dtype), requires_grad=True)
            y = Variable(y_.type(dtype), requires_grad=False).long()

            logits = model(X)
            loss = F.cross_entropy(logits, y)
            loss.backward()

            with torch.no_grad():
                X += X.grad.sign() * eps
                X.grad.zero_()

            X.requires_grad = False
            X = (X * 255).long().float() / 255

            logits = model(X)
            _, preds = logits.max(1)

            num_correct += (preds == y).sum()
            num_samples += preds.size(0)

        accuracy = float(num_correct) / num_samples * 100
        print('\nAttack using FGSM with eps = %.3f, accuracy = %.2f%%' % (eps, accuracy))

## Main Function

In [9]:
loader_train, loader_test = loadData()
dtype = torch.cuda.FloatTensor

# Epochs, Epsilon, Loss Iterations, Loss Learning rate
# settings = [[2], [0.1, 0.2, 0.3, 0.4], [5, 10, 15, 20], [0.1, 0.2, 0.3, 0.4]]

settings1 = [[5], [0.4], [10], [0.1, 0.2, 0.3, 0.4]]
settings2 = [[5], [0.4], [5, 10, 15, 20], [0.1]]
settings3 = [[5], [0.1, 0.2, 0.3, 0.4], [10], [0.1]]

for sett in itertools.product(*settings1):
    model, loss_list = train(loader_train, loader_test, dtype, sett)
    pgdAttackTest(model, loader_test, dtype)
    fgsmAttackTest(model, loader_test, dtype)
    plt.plot(loss_list, label=f"{sett[3]}")
plt.legend()
plt.savefig(f"Graphs/learning_rate.png")
plt.clf()

for sett in itertools.product(*settings2):
    model, loss_list = train(loader_train, loader_test, dtype, sett)
    pgdAttackTest(model, loader_test, dtype)
    fgsmAttackTest(model, loader_test, dtype)
    plt.plot(loss_list, label=f"{sett[2]}")
plt.legend()
plt.savefig(f"Graphs/iterations.png")
plt.clf()
for sett in itertools.product(*settings3):
    model, loss_list = train(loader_train, loader_test, dtype, sett)
    pgdAttackTest(model, loader_test, dtype)
    fgsmAttackTest(model, loader_test, dtype)
    plt.plot(loss_list, label=f"{sett[1]}")
plt.legend()
plt.savefig(f"Graphs/epsilon.png")
plt.clf()
    
# fname = "./Model/model.pth"
# torch.save(model, fname)
# print("Training done, model save to %s" % fname)

# fname = "./Model/model.pth"
# model = torch.load(fname)



Training 5 epochs with learning rate 0.0005000

Training epoch 1 / 5 ...

Batch 100 done, loss = 2.3142447

Accuracy = 35.42%
Batch 200 done, loss = 2.3009474

Accuracy = 17.31%
Batch 300 done, loss = 2.3019876

Accuracy = 11.35%
Batch 400 done, loss = 2.3067305

Accuracy = 11.35%
Batch 500 done, loss = 2.2998536

Accuracy = 11.35%
Batch 600 done, loss = 2.3056114

Accuracy = 34.35%
Batch 700 done, loss = 2.3123944

Accuracy = 24.89%
Batch 800 done, loss = 2.3014607

Accuracy = 28.98%
Batch 900 done, loss = 2.3006124

Accuracy = 42.36%
Batch 938 done, loss = 2.2880611

Training epoch 2 / 5 ...

Batch 100 done, loss = 2.2438798

Accuracy = 60.09%
Batch 200 done, loss = 2.2423337

Accuracy = 64.66%
Batch 300 done, loss = 2.2512317

Accuracy = 69.49%
Batch 400 done, loss = 2.1779463

Accuracy = 67.23%
Batch 500 done, loss = 2.1659908

Accuracy = 82.55%
Batch 600 done, loss = 2.2053752

Accuracy = 75.09%
Batch 700 done, loss = 2.1408150

Accuracy = 80.11%
Batch 800 done, loss = 2.0420349


<Figure size 432x288 with 0 Axes>